In [4]:
import pandas as pd
from matplotlib import rcParams
import seaborn as sns
import numpy as np
from IPython.display import display
rcParams.update({'figure.autolayout': True})
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
import sys
import collections
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles, venn3_unweighted
import ipywidgets as widgets
from ipywidgets import interact, fixed
from Bio.Data.IUPACData import ambiguous_dna_values
plt.tight_layout()


Read in data that was generated by galaxy team and EC-19 team along with the table from Ryan that has the metadata.

In [5]:
srr_mdata = pd.read_csv("../data/ONT_Illumina_TRACE_test_set.csv")
galaxy_df = pd.read_csv("../data/galaxy.csv", sep=",")
ec19_df = pd.read_csv("../data/ONT_illumina_TRACE_test_result_with_metadata.txt", sep="\t")

In [28]:
def disp_ec19srr(srr):
    display(ec19_df[ec19_df['Accession'] == srr][['Group_Name', 'Accession', 'Pos', 'Ref', 'Alt', 'DP', 'G_AD_2', 'Alt_Freq']])
interact(disp_ec19srr, srr=list(set(ec19_df['Accession'].to_list())));

interactive(children=(Dropdown(description='srr', options=('ERR4970058', 'ERR5514445', 'ERR4969191', 'ERR49693…

In [25]:
def disp_galaxysrr(srr):
    display(galaxy_df[galaxy_df['Accession'] == srr])
interact(disp_srr, srr=list(set(galaxy_df['Accession'].to_list())));
# list(set(ec19_df['Accession'].to_list()))

interactive(children=(Dropdown(description='srr', options=('ERR4970058', 'ERR4969191', 'ERR4969320', 'SRR13870…

<Figure size 640x480 with 0 Axes>

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

Add a column to galaxy dataframe for naming if its snp, in, del, or multi-snp

In [33]:
def galaxy_status(row):
    if len(row['Ref']) > len(row['Alt']) :
        return "deletion"
    elif len(row['Ref']) < len(row['Alt']) :
        return "insertion"
    elif len(row['Ref']) == len(row['Alt']):
        if len(row['Ref']) == 1 and len(row['Alt']) == 1:
            if row['Ref'] != row['Alt']:
                return "snp"
        elif len(row['Ref']) > 1 and len(row['Alt']) > 1:
            return "multi-snp"
galaxy_df['gal_status'] = galaxy_df.apply(galaxy_status, axis=1)

#=================================================================
galaxy_multisnp = pd.DataFrame()
for x, row in galaxy_df[galaxy_df['gal_status']=="multi-snp"].iterrows():
    pos = row['Pos']
    mut_pos = 0
    for nuc in row['Ref']:
        galaxy_multisnp = galaxy_multisnp.append({'Group_Name':'Galaxy',
                        'Accession':row['Accession'],
                        "Pos":pos, "Ref":row['Ref'][mut_pos],
                        "Alt":row['Alt'][mut_pos], "DP":row['DP'],
                        "G_AD_2":row["G_AD_2"],	"Alt_Freq":row["Alt_Freq"], "gal_status":row["gal_status"]}, ignore_index=True)
        pos = pos + 1
        mut_pos = mut_pos + 1


print(galaxy_multisnp)



       Accession Alt  Alt_Freq      DP  G_AD_2 Group_Name      Pos Ref  \
0     ERR4365002   A  0.983502  1840.0  1848.0     Galaxy  28881.0   G   
1     ERR4365002   A  0.983502  1840.0  1848.0     Galaxy  28882.0   G   
2     ERR4365002   C  0.983502  1840.0  1848.0     Galaxy  28883.0   G   
3     ERR4365199   A  0.983091  5714.0  5756.0     Galaxy  28881.0   G   
4     ERR4365199   A  0.983091  5714.0  5756.0     Galaxy  28882.0   G   
..           ...  ..       ...     ...     ...        ...      ...  ..   
231  SRR13402841   T  0.994340   459.0   527.0     Galaxy  28281.0   A   
232  SRR13402841   A  0.994340   459.0   527.0     Galaxy  28282.0   T   
233  SRR13402841   A  0.985714    67.0    69.0     Galaxy  28881.0   G   
234  SRR13402841   A  0.985714    67.0    69.0     Galaxy  28882.0   G   
235  SRR13402841   C  0.985714    67.0    69.0     Galaxy  28883.0   G   

    gal_status  
0    multi-snp  
1    multi-snp  
2    multi-snp  
3    multi-snp  
4    multi-snp  
..       

Expand deletion

In [34]:
galaxy_del = pd.DataFrame()
for x, row in galaxy_df[galaxy_df['gal_status']=="deletion"].iterrows():
    pos = row['Pos']
    del_pos = 0
    if row["Ref"][0] == row["Alt"]:
        galaxy_del = galaxy_del.append({'Group_Name':'Galaxy',
                            'Accession':row['Accession'],
                            "Pos":pos+1, "Ref":row['Ref'][1:],
                            "Alt":'-', "DP":row['DP'],
                            "G_AD_2":row["G_AD_2"],	"Alt_Freq":row["Alt_Freq"], "gal_status":row["gal_status"]}, ignore_index=True)
    else:
        galaxy_del = galaxy_del.append({'Group_Name':'Galaxy',
                                'Accession':row['Accession'],
                                "Pos":pos, "Ref":row['Ref'][0],
                                "Alt":'-', "DP":row['DP'],
                                "G_AD_2":row["G_AD_2"],	"Alt_Freq":row["Alt_Freq"], "gal_status":"snp"}, ignore_index=True)
        galaxy_del = galaxy_del.append({'Group_Name':'Galaxy',
                                'Accession':row['Accession'],
                                "Pos":pos+1, "Ref":row['Ref'][1:],
                                "Alt":'-', "DP":row['DP'],
                                "G_AD_2":row["G_AD_2"],	"Alt_Freq":row["Alt_Freq"], "gal_status":row["gal_status"]}, ignore_index=True)
        print(row)
        print(row["Ref"][0] , row["Alt"])
galaxy_del


Group_Name        Galaxy
Accession     ERR4969365
Pos                27395
Ref                   TG
Alt                    A
DP                   619
G_AD_2               868
Alt_Freq        0.976378
gal_status      deletion
Name: 3009, dtype: object
T A


,Accession,Alt,Alt_Freq,DP,G_AD_2,Group_Name,Pos,Ref,gal_status
0,ERR4364963,-,0.196552,290.0,57.0,Galaxy,27098.0,T,deletion
1,ERR4365125,-,0.547508,1284.0,703.0,Galaxy,6776.0,TA,deletion
2,ERR4365125,-,0.288028,18526.0,5336.0,Galaxy,7910.0,TG,deletion
3,ERR4365125,-,0.216749,203.0,44.0,Galaxy,9251.0,G,deletion
4,ERR4365125,-,0.162455,277.0,45.0,Galaxy,9267.0,GTG,deletion
...,...,...,...,...,...,...,...,...,...
203,SRR13402841,-,0.963415,7.0,79.0,Galaxy,11288.0,TCTGGTTTT,deletion
204,SRR13402841,-,0.934783,37.0,258.0,Galaxy,21765.0,TACATG,deletion
205,SRR13402841,-,0.897059,79.0,122.0,Galaxy,21991.0,TTA,deletion
206,SRR13402841,-,0.930233,57.0,480.0,Galaxy,28271.0,A,deletion


Concatenating dataframes

In [35]:
galaxy_df_final = pd.concat([galaxy_del, galaxy_multisnp, galaxy_df[~galaxy_df['gal_status'].isin(['deletion', 'multi-snp'])] ])
galaxy_df_final

,Accession,Alt,Alt_Freq,DP,G_AD_2,Group_Name,Pos,Ref,gal_status
0,ERR4364963,-,0.196552,290.0,57.0,Galaxy,27098.0,T,deletion
1,ERR4365125,-,0.547508,1284.0,703.0,Galaxy,6776.0,TA,deletion
2,ERR4365125,-,0.288028,18526.0,5336.0,Galaxy,7910.0,TG,deletion
3,ERR4365125,-,0.216749,203.0,44.0,Galaxy,9251.0,G,deletion
4,ERR4365125,-,0.162455,277.0,45.0,Galaxy,9267.0,GTG,deletion
...,...,...,...,...,...,...,...,...,...
4792,SRR13870639,G,0.929752,269.0,225.0,Galaxy,18082.0,A,snp
4793,SRR13870639,G,0.843844,324.0,281.0,Galaxy,23403.0,A,snp
4794,SRR13870639,T,0.933168,368.0,377.0,Galaxy,25563.0,G,snp
4795,SRR13870639,G,0.806738,551.0,455.0,Galaxy,27357.0,A,snp


Add the status to ec19 dataframe

In [36]:
def ec19_status(row):
    if len(row['Ref']) > len(row['Alt']):
        if row['Alt'] == "-":
            return "deletion"
        else:
            return None
    elif len(row['Ref']) < len(row['Alt']):
        if '(' in row['Alt']:
            return "snp"
        else:
            return "insertion"
    elif len(row['Ref']) == len(row['Alt']):
        if len(row['Ref']) == 1 and len(row['Alt']) == 1:
            if row['Ref'] != row['Alt']:
                return "snp"
        elif len(row['Ref']) > 1 and len(row['Alt']) > 1:
            return "multi-snp"
        else:
            return None
    else:
        return None
                
    
    # elif len(row['Ref']) == 1 and len(row['Alt']) == 1:
ec19_df['ec19_status'] = ec19_df.apply(ec19_status, axis=1)
ec19_df[ec19_df['ec19_status'] == "deletion"][['Group_Name',	'Accession',	'Pos',	'Ref',	'Alt', 'DP', "G_AD_2",	"Alt_Freq", "ec19_status"]]
# ec19_df[ec19_df['Accession'] == "ERR4364963"][['Group_Name',	'Accession',	'Pos',	'Ref',	'Alt', 'DP', "G_AD_2",	"Alt_Freq"]]

,Group_Name,Accession,Pos,Ref,Alt,DP,G_AD_2,Alt_Freq,ec19_status
17,LANL,ERR4365125,6776,TA,-,1116,599,0.5376,deletion
221,LANL,ERR5514445,6655,TAA,-,1714,945,0.5519,deletion
273,LANL,ERR5627824,21765,TACATG,-,42,42,1.0000,deletion
300,LANL,ERR5628809,11288,TCTGGTTTT,-,2394,2354,0.9833,deletion
305,LANL,ERR5628809,21765,TACATG,-,458,454,0.9913,deletion
...,...,...,...,...,...,...,...,...,...
7479,LANL,SRR12480567,11075,TT,-,129,88,0.6822,deletion
7510,LANL,SRR12447371,11075,TT,-,147,102,0.6939,deletion
8186,LANL,SRR13870634,11075,TT,-,6711,4182,0.6233,deletion
8198,LANL,SRR13870635,11075,TT,-,6312,3929,0.6225,deletion


Add a column that combines SRR and the position for each column.

In [38]:
galaxy_df_final['galaxy_vars'] = galaxy_df_final['Accession'].astype(str) + "_" + galaxy_df_final['Pos'].astype(str)
ec19_df['ec19_vars'] = ec19_df['Accession'].astype(str) + "_" + ec19_df['Pos'].astype(str) + ".0"
# sync_galaxy_df = galaxy_df_final[galaxy_df_final['Accession'].isin(ec19_df['Accession'].to_list())]
# sync_galaxy_df = sync_galaxy_df.merge(srr_mdata, how="left", left_on="Accession", right_on="acc")
# sync_ec19_df = ec19_df[ec19_df['Accession'].isin(galaxy_df_final['Accession'].to_list())]
# galaxy_df_final



# sync_galaxy_df.head()

# sync_galaxy_df.head()
# sync_ec19_df.head()
# ec19_df[['Pos','ec19_vars']].head()
ec19_df[['Group_Name',	'Accession',	'Pos',	'Ref',	'Alt', 'DP', "G_AD_2",	"Alt_Freq", "ec19_status", "ec19_vars"]]
# galaxy_df_final

,Group_Name,Accession,Pos,Ref,Alt,DP,G_AD_2,Alt_Freq,ec19_status,ec19_vars
0,LANL,ERR4364963,241,C,T,3510,3441,0.9806,snp,ERR4364963_241.0
1,LANL,ERR4364963,3037,C,T,1752,1750,0.9994,snp,ERR4364963_3037.0
2,LANL,ERR4364963,4186,T,C,6464,6462,0.9998,snp,ERR4364963_4186.0
3,LANL,ERR4364963,11801,G,A,6513,6511,0.9997,snp,ERR4364963_11801.0
4,LANL,ERR4364963,14408,C,T,6418,6414,0.9995,snp,ERR4364963_14408.0
...,...,...,...,...,...,...,...,...,...,...
8243,LANL,SRR13870639,23874,C,C(Y),1968,515,0.2617,snp,SRR13870639_23874.0
8244,LANL,SRR13870639,25563,G,T,7268,6600,0.9081,snp,SRR13870639_25563.0
8245,LANL,SRR13870639,27357,A,G,7847,6633,0.8453,snp,SRR13870639_27357.0
8246,LANL,SRR13870639,27964,C,T,4417,4008,0.9076,snp,SRR13870639_27964.0


Expand IUPAC code to nuleotides

In [42]:
def expand_amb(row):
    if '(' in row['Alt']:
        iupac = row['Alt'].split('(')[1].replace(')', '')
        return ambiguous_dna_values[iupac]
    else:
        return row['Alt']
ec19_df['New_Alt'] = ec19_df.apply(expand_amb, axis=1)
ec19_df

gal_ec19_df = galaxy_df_final[['Ref','Alt', 'galaxy_vars']].merge(ec19_df[['Ref','ec19_vars','New_Alt']], left_on="galaxy_vars", right_on="ec19_vars", how="outer")
gal_ec19_df.columns = ["galaxy_ref", "galaxy_alt", "galaxy_pos", "ec19_ref", "ec19_pos", "ec19_alt"]
# gal_ec19_df['match'] = gal_ec19_df.apply(check_match, axis=1)
gal_ec19_df


,galaxy_ref,galaxy_alt,galaxy_pos,ec19_ref,ec19_pos,ec19_alt
0,T,-,ERR4364963_27098.0,NaN,NaN,NaN
1,TA,-,ERR4365125_6776.0,TA,ERR4365125_6776.0,-
2,TG,-,ERR4365125_7910.0,NaN,NaN,NaN
3,G,-,ERR4365125_9251.0,NaN,NaN,NaN
4,GTG,-,ERR4365125_9267.0,NaN,NaN,NaN
...,...,...,...,...,...,...
8885,NaN,NaN,NaN,G,SRR13870639_2198.0,AG
8886,NaN,NaN,NaN,C,SRR13870639_5736.0,CT
8887,NaN,NaN,NaN,T,SRR13870639_11075.0,-
8888,NaN,NaN,NaN,C,SRR13870639_23874.0,CT


In [60]:
def check_match(row):
    # print(row)
    if str(row['galaxy_alt']) in str(row['ec19_alt']):
        return True
    elif pd.isna(row['galaxy_ref']): # SNP/indel in EC-19, not in galaxy
        return False
    elif pd.isna(row['ec19_ref']): #SNP/indel in galaxy, not in EC-19
        return False
    elif pd.isna(row['ec19_alt']) is None: #
        # print(row)
        if row['galaxy_alt'] == "-":
            return False
    
gal_ec19_df['match'] = gal_ec19_df.apply(check_match, axis=1)
gal_ec19_df[gal_ec19_df['match'] == False].to_csv("test.txt")

In [ ]:
sync_galaxy_ont = sync_galaxy_df[sync_galaxy_df['platform'] == "OXFORD_NANOPORE"]
sync_galaxy_ill = sync_galaxy_df[sync_galaxy_df['platform'] == "ILLUMINA"]
sync_ec19_ont= sync_ec19_df[sync_ec19_df['platform'] == "OXFORD_NANOPORE"]
sync_ec19_ill= sync_ec19_df[sync_ec19_df['platform'] == "ILLUMINA"]
sync_ec19_ill[['Group_Name',	'Accession',	'Pos',	'Ref',	'Alt', 'DP', "G_AD_2",	"Alt_Freq"]].head()

In [ ]:
gal_ec19_df[gal_ec19_df['ec19_ref'] == "ERR4364963_27097"]
gal_ec19_df[gal_ec19_df['match'] == True]

In [ ]:
def check_overlap(row):
    if collections.Counter(row['galaxy']) == collections.Counter(row['ec19']):
        return True
    else:
        return False

def find_unique(list1):
    gx_pos = row['galaxy'].to_list()
    ec19_pos = row['ec19'].to_list()
    gx_unq = np.setdiff1d(gx_pos,ec19_pos)
    ec19_unq = np.setdiff1d(ec19_pos, gx_pos)
    return gx_unq

def merge_gx_ec19(galaxy_df, ec19_df):
    ec19_gx_ont = pd.concat([galaxy_df[['Group_Name', 'Accession', 'Pos']], ec19_df[['Group_Name', 'Accession', 'Pos']]], ignore_index=True)
    gx_all_pos = galaxy_df.groupby(['Accession'])['Pos'].apply(list).reset_index()
    ec19_all_pos = ec19_df.groupby(['Accession'])['Pos'].apply(list).reset_index()
    sync_all_pos = ec19_all_pos.merge(gx_all_pos, on="Accession")
    sync_all_pos.columns = ["acc", "galaxy", "ec19"]
    return sync_all_pos

def compare_srrs(merged_df):
    merged_df['overlapped'] = merged_df.apply(check_overlap, axis=1)
    return merged_df.groupby('overlapped').agg('count').reset_index()

ont_comp_df = merge_gx_ec19(sync_ec19_ont, sync_galaxy_ont)
ill_comp_df= merge_gx_ec19(sync_ec19_ill, sync_galaxy_ill)
ill_comp_df
# compare_srrs(ont_comp_df)
# compare_srrs(ill_comp_df)
# ont_comp_df['galaxy_unq'] = ont_comp_df.apply(lambda row: np.setdiff1d(row["galaxy"], row["ec19"]), axis=1)
# ont_comp_df['ec19_unq'] = ont_comp_df.apply(lambda row: np.setdiff1d(row["ec19"], row["galaxy"]), axis=1)
# ill_comd_df

In [ ]:
sync_ec19_ill['Alt'].to_list()

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
sns.set_theme(style="whitegrid")
ax=sns.barplot(data=compare_srrs(ont_comp_df), x="overlapped", y="acc")
ax.set_title('OXFORD NANOPORE')
plt.subplot(1,2,2)
sns.set_theme(style="whitegrid")
ax=sns.barplot(data=compare_srrs(ill_comp_df), x="overlapped", y="acc")
ax.set_title('ILLUMINA')
plt.show()

In [ ]:
ill_comd_df

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
venn2_unweighted([set(sync_galaxy_ill['galaxy_vars'].to_list()), set(sync_ec19_ill['ec19_vars'].to_list())], set_labels = ('galaxy', 'EC-19'), alpha = 0.5)
plt.title("OXFORD NANOPORE")
plt.subplot(1, 2, 2)
venn2_unweighted([set(sync_galaxy_ont['galaxy_vars'].to_list()), set(sync_ec19_ont['ec19_vars'].to_list())], set_labels = ('galaxy', 'EC-19'), alpha = 0.5)
plt.title("ILLUMINA")
plt.show()
